[스팸 수집]
- selenium 이용 로그인 - 쿠키 생성
- Requests.session으로 쿠키 전달
- 쿠키를 이용, 메일(제목) 스크래핑(0:받은편지함, 5:스팸메일함) => json to dict, ['mailList'] 참조

[학습]
- 클래스별(스팸, 정상) 문서 구분
- 각 문서 내 단어(전처리-색인어 추출) 빈도 계산
- N(문서갯수), Nc(클래스 별 문서 갯수)를 이용한 클래스의 사전확률 계산(Add-one/Laplace Smoothing 적용)
- 모든 단어(V)에 대한 각 Nc 내 단어(Tc)의 조건부 확률계산

[분류]
- 스팸과 정상 메일에 대한 분류 성능 테스트
- 대상 메일 내 단어 추출
- 단어 별 학습된 조건부확률과 클래스별 사전 확률의 log 합을 구해 판단

[Tip]
- 깨질 경우 import html, html.parser.unescape(문자열) 사용

[kNN 문서분류를 Naive Bayes로 변경하여 분류기 만들기]

In [292]:
from selenium import webdriver

driver = webdriver.Chrome()

In [293]:
url = "https://nid.naver.com/nidlogin.login?url=http%3A%2F%2Fmail.naver.com%2F"
driver.get(url)

# 창이 뜬 후 로그인은 사람이 수행.

In [294]:
import requests
session = requests.Session() # get이 든 post는 이 session을 통해 컨트롤할 예정
# driver.get_cookies() : dictionary
for cookie in driver.get_cookies():
    session.cookies.set(cookie["name"], cookie["value"]) # 셀레니움에서 받은 쿠기가 들어옴. 

In [295]:
driver.close() # 더이상 셀레니움 필요 없음

In [296]:
url = "https://mail.naver.com/json/list"

data_normalmail = {
    "page":1, 
    "sortField":1,
    "sortType":0,
    "folderSN":0,
    "isUnread":"false",
    "u":"brstar96",
}
data_spammail = {
    "page":1, 
    "sortField":1,
    "sortType":0,
    "folderSN":5,
    "isUnread":"false",
    "u":"brstar96",
}

resp_normalmail = session.post(url, data_normalmail) # response 객체 생성
resp_normalmail.text # 네이버는 틀은 그대로 둔 채 주기적으로 자바스크립트가 돌며 서버로부터 메일과 관련한 내용을 체크해 받아옴.
resp_spammail = session.post(url, data_spammail) # response 객체 생성
resp_spammail.text # 네이버는 틀은 그대로 둔 채 주기적으로 자바스크립트가 돌며 서버로부터 메일과 관련한 내용을 체크해 받아옴.

'{"currentTime":1554467782,"lastPage":2,"Message":"","pageSize":15,"unreadCount":14,"mailData":[{"mailSN":32997,"folderSN":5,"folderName":null,"iconType":0,"status":2097152,"from":{"name":"쇼킹딜","email":"11st@ems.11st.co.kr"},"subject":"(광고)단하루 대박행사! 액츠 액체 세탁세제 선착순 한정특가!","receivedTime":1554459766,"sentTime":1554459766,"size":131811,"attachCount":0,"iDomainEmail":"brstar96@naver.com","priority":3,"securityLevel":null,"reSend":false,"color":"#ffffff","preview":"","mimeSN":"1554459766.318609.6269.1792","toList":[{"name":"이명규님","email":"brstar96@naver.com"}],"spamType":null,"attachInfo":[],"threadId":"","bccList":[],"receiveByBcc":false,"threadCount":0,"unreadThreadCount":0,"firstLocatedTime":1554459766,"remindAt":0,"threadRemindAt":0,"threadRemindFlag":0,"category":213,"bigFile":false,"toMe":true,"statusHex":"200000","isDomainMail":false},{"mailSN":32992,"folderSN":5,"folderName":null,"iconType":0,"status":2105344,"from":{"name":"와디즈 투자 뉴스레터","email":"newsletter@wadiz.kr"},"subject":"(광고)

In [297]:
import json

dict_normalmail = json.loads(resp_normalmail.text)
dict_spammail = json.loads(resp_spammail.text)

dict_normalmail

{'currentTime': 1554467782,
 'lastPage': 29,
 'Message': '',
 'pageSize': 15,
 'unreadCount': 0,
 'mailData': [{'mailSN': 32996,
   'folderSN': 0,
   'folderName': None,
   'iconType': 0,
   'status': 2097157,
   'from': {'name': '이우리', 'email': 'syaeom@ssu.ac.kr'},
   'subject': '랩실 대표연락 메일 조사 및 연구실 상주 인원 조사요청(4.8월 17:00까지)',
   'receivedTime': 1554454207,
   'sentTime': 1554447925,
   'size': 372378,
   'attachCount': 1,
   'iDomainEmail': 'brstar96@naver.com',
   'priority': 3,
   'securityLevel': None,
   'reSend': False,
   'color': '#ffffff',
   'preview': '',
   'mimeSN': '1554454207.917936.64474.34560',
   'toList': [{'name': '', 'email': 'harkkwon2@naver.com'}],
   'spamType': None,
   'attachInfo': [],
   'threadId': '<5ca6fe473fec_@_imoxion.com>',
   'bccList': [],
   'receiveByBcc': False,
   'threadCount': 0,
   'unreadThreadCount': 0,
   'firstLocatedTime': 1554454207,
   'remindAt': 0,
   'threadRemindAt': 0,
   'threadRemindFlag': 0,
   'category': -1,
   'bigFile': Fal

In [298]:
normalmailList = []
spammailList = []

for row in dict_normalmail['mailData']:
    normalmailList.append(row['subject'])
for row in dict_spammail['mailData']:
    spammailList.append(row['subject'])

print("-> normalmailList : ", "\n")
for i in range(len(normalmailList)):
    print(normalmailList[i])
print("\n")
print("-> spammailList : ","\n")
for i in range(len(spammailList)):
    print(spammailList[i])

-> normalmailList :  

랩실 대표연락 메일 조사 및 연구실 상주 인원 조사요청(4.8월 17:00까지)
한국표준과학연구원 포닥 추천 의뢰
[T아카데미] 토크ON세미나(50차) &quot;Scikit-learn으로 다지는 머신러닝 기초&quot; 안내
Re: [AiHackathon2018/AI-Vision] Clova vision팀 논문 공유드립니다. (#314)
[DSC2019] 대회 관련 안내
[Data Science Competition 2019] 온라인 강의 추천 안내
KCGS 2019 학술대회 일정 및 장소 공지
opic 자료
NAVER AI Colloquium 2019 결과 안내
NAVER AI Colloquium 2019 의 참가신청이 완료되었습니다.
[T아카데미] 토크ON세미나(48차) &#39;Kaggle(캐글) 데이터 분석 첫걸음&#39; 신청 안내
RE: [LINE] 안녕하세요, 라인 채용담당자 이선주 입니다.
[LINE] 안녕하세요, 라인 채용담당자 이선주 입니다.
연구윤리 교육과정 신청 명단 양식입니다.
[DSC2019] 지원서 접수 URL 입니다.


-> spammailList :  

(광고)단하루 대박행사! 액츠 액체 세탁세제 선착순 한정특가!
(광고) 명품 브랜드의 러브콜을 부른 기업
Myeong-gyu.Lee 님, 혼자만 보기 아까운 트윗을 공유해 보세요.
(광고)사업자 신규회원 전용! 30%쿠폰 2종 쿠폰 혜택!
(광고)오늘 단하루 커피머신 모음전 파격특가! 119,000원~
[비트맨 빠른NEWS] 04/04 &quot;비트코인, 전문가 꼽은 3가지 급등 이유&quot;
(광고)CJ 햇반 210*6입*4개 특가 24,900원~
기즈모입니다. 소니 플래그쉽 제품 특별 공동구매
(광고)오늘 단하루! 마미포코 에어핏 10%+25% 쿠폰 혜택+사은품
이번 주 셰에라자드 소식 전해드립니다!
AK카페에서 SACD 앨범 공동구매를 시작했습니다.
(광고)오늘 단 하루! 아비노 데일리 로션 40%쿠폰 혜택 21,900원~
트

In [439]:
# 전처리 및 색인어 추출
import re
from string import punctuation


def getPatternList():
    patternList = dict()
    pattern = re.compile(r"[%s]{1,}" % re.escape(punctuation)) # punctuation 안의 특수문자가 두번이상 반복되는 모든 문자에 대해 패턴 정의
    patternList["Punctuation"] = pattern
    pattern2 = re.compile(r"([A-Za-z0-9\-\_\.]{3,}@[A-Za-z0-9\-\_]{3,}(.[A-Za-z]{2,})+)") #이메일주소제거패턴
    patternList["Email"] = pattern2
    pattern3 = re.compile(r"([A-Za-z0-9\-\_]{1,}(.[A-Za-z]{2,})+)") # 신문사도메인패턴
    patternList["Domain"] = pattern3
    pattern4 = re.compile(r"\s{2,}") # 두칸짜리 공백제거
    patternList["Whitespace"] = pattern4
    pattern5 = re.compile(r"([^ㄱ-ㅎㅏ-ㅣ가-힣0-9]+)") # 한글이 아닌 영어 기호 제거
    patternList["Korean"] = pattern5
    pattern6 = re.compile(r'[A-Za-z-_]{8,}') # 영어 (대소문자) +\-\_ 제거 (8글자 이상)
    patternList["ElimLongEng"] = pattern6
    pattern7 = re.compile(r"quot|Myeong|gyu|Lee")
    patternList["ElimRecWord"] = pattern7
                          
    return patternList

def Elimpunc(mailList):
    patternList = getPatternList()
    for _ in ["Whitespace", "Punctuation", "ElimRecWord"]: 
        for i in range(len(mailList)):
            mailList[i] = patternList[_].sub(" ", mailList[i])
    return mailList

In [440]:
print("-> 전처리된 normalmailList : ", Elimpunc(normalmailList), "\n")
print("-> 전처리된 spammailList : ", Elimpunc(spammailList))

-> 전처리된 normalmailList :  ['랩실 대표연락 메일 조사 및 연구실 상주 인원 조사요청 4 8월 17 00까지 ', '한국표준과학연구원 포닥 추천 의뢰', ' T아카데미 토크ON세미나 50차 Scikit learn으로 다지는 머신러닝 기초 안내', 'Re AiHackathon2018 AI Vision Clova vision팀 논문 공유드립니다 314 ', ' DSC2019 대회 관련 안내', ' Data Science Competition 2019 온라인 강의 추천 안내', 'KCGS 2019 학술대회 일정 및 장소 공지', 'opic 자료', 'NAVER AI Colloquium 2019 결과 안내', 'NAVER AI Colloquium 2019 의 참가신청이 완료되었습니다 ', ' T아카데미 토크ON세미나 48차 39 Kaggle 캐글 데이터 분석 첫걸음 39 신청 안내', 'RE LINE 안녕하세요 라인 채용담당자 이선주 입니다 ', ' LINE 안녕하세요 라인 채용담당자 이선주 입니다 ', '연구윤리 교육과정 신청 명단 양식입니다 ', ' DSC2019 지원서 접수 URL 입니다 '] 

-> 전처리된 spammailList :  [' 광고 단하루 대박행사 액츠 액체 세탁세제 선착순 한정특가 ', ' 광고 명품 브랜드의 러브콜을 부른 기업', ' 님 혼자만 보기 아까운 트윗을 공유해 보세요 ', ' 광고 사업자 신규회원 전용 30 쿠폰 2종 쿠폰 혜택 ', ' 광고 오늘 단하루 커피머신 모음전 파격특가 119 000원 ', ' 비트맨 빠른NEWS 04 04 비트코인 전문가 꼽은 3가지 급등 이유 ', ' 광고 CJ 햇반 210 6입 4개 특가 24 900원 ', '기즈모입니다 소니 플래그쉽 제품 특별 공동구매', ' 광고 오늘 단하루 마미포코 에어핏 10 25 쿠폰 혜택 사은품', '이번 주 셰에라자드 소식 전해드립니다 ', 'AK카페에서 SACD 앨범 공동구매를 시작했습니다 ', ' 광고 오늘 단 하루 아비노 데일리 로션 40 쿠폰

### 학습데이터 정의

In [441]:
trainingSet = []
# print(len(normalmailList)+len(spammailList))

for i in range(len(normalmailList)+len(spammailList)):
    if i<15:
#         trainingset.append(3)
        trainingSet.append((i, str(normalmailList[i]), "yes"),)
    elif 14 < i <30:
        trainingSet.append((i, str(spammailList[i-15]), "no"))
        
for i in range(len(trainingSet)):
    print(trainingSet[i])
        
testSet = [
    (30, "(광고) [대학생 복지소식] 삼성 프리도스 노트북 특가 마지막주!", "spam"), # 정답 : 스팸메일 
    (31, "(광고) 신용걱정 끝! 최저금리와 높은승인률로 '최대8천만원'까지 믿음직한 서민신용대출!!", "spam"), # 정답 : 스팸메일
    (32, "200% 정품 비아그라 시알리스 / 온라인 최저가 판매 / 정품아닐시 200% 보상", "spam"), # 정답 : 스팸메일
    (33, "케라스 & 캐글 전국콘서트 서울 발표자료 송부", "normal"), # 정답 : 정상메일
    (34, "RE: [NAVER] AI 해커톤_AI Vision 오프라인 결선 리마인드", "normal"), # 정답 : 정상메일
    (35, "[한국IT비즈니스진흥협회]인공지능센터 고급	과정(저녁) 교육생 선정안내", 'normal') # 정답 : 정상메일
    
]

(0, '랩실 대표연락 메일 조사 및 연구실 상주 인원 조사요청 4 8월 17 00까지 ', 'yes')
(1, '한국표준과학연구원 포닥 추천 의뢰', 'yes')
(2, ' T아카데미 토크ON세미나 50차 Scikit learn으로 다지는 머신러닝 기초 안내', 'yes')
(3, 'Re AiHackathon2018 AI Vision Clova vision팀 논문 공유드립니다 314 ', 'yes')
(4, ' DSC2019 대회 관련 안내', 'yes')
(5, ' Data Science Competition 2019 온라인 강의 추천 안내', 'yes')
(6, 'KCGS 2019 학술대회 일정 및 장소 공지', 'yes')
(7, 'opic 자료', 'yes')
(8, 'NAVER AI Colloquium 2019 결과 안내', 'yes')
(9, 'NAVER AI Colloquium 2019 의 참가신청이 완료되었습니다 ', 'yes')
(10, ' T아카데미 토크ON세미나 48차 39 Kaggle 캐글 데이터 분석 첫걸음 39 신청 안내', 'yes')
(11, 'RE LINE 안녕하세요 라인 채용담당자 이선주 입니다 ', 'yes')
(12, ' LINE 안녕하세요 라인 채용담당자 이선주 입니다 ', 'yes')
(13, '연구윤리 교육과정 신청 명단 양식입니다 ', 'yes')
(14, ' DSC2019 지원서 접수 URL 입니다 ', 'yes')
(15, ' 광고 단하루 대박행사 액츠 액체 세탁세제 선착순 한정특가 ', 'no')
(16, ' 광고 명품 브랜드의 러브콜을 부른 기업', 'no')
(17, ' 님 혼자만 보기 아까운 트윗을 공유해 보세요 ', 'no')
(18, ' 광고 사업자 신규회원 전용 30 쿠폰 2종 쿠폰 혜택 ', 'no')
(19, ' 광고 오늘 단하루 커피머신 모음전 파격특가 119 000원 ', 'no')
(20, ' 비트맨 빠른NEWS 04 04 비트코인 전문가 꼽은 3가지 급등 이유 ', 'no')
(21, ' 

In [442]:
trainingSetYes = [d for d in trainingSet if d[2] == "yes"]
trainingSetNo = [d for d in trainingSet if d[2] == "no"]

In [443]:
trainingSetYes, trainingSetNo

([(0, '랩실 대표연락 메일 조사 및 연구실 상주 인원 조사요청 4 8월 17 00까지 ', 'yes'),
  (1, '한국표준과학연구원 포닥 추천 의뢰', 'yes'),
  (2, ' T아카데미 토크ON세미나 50차 Scikit learn으로 다지는 머신러닝 기초 안내', 'yes'),
  (3, 'Re AiHackathon2018 AI Vision Clova vision팀 논문 공유드립니다 314 ', 'yes'),
  (4, ' DSC2019 대회 관련 안내', 'yes'),
  (5, ' Data Science Competition 2019 온라인 강의 추천 안내', 'yes'),
  (6, 'KCGS 2019 학술대회 일정 및 장소 공지', 'yes'),
  (7, 'opic 자료', 'yes'),
  (8, 'NAVER AI Colloquium 2019 결과 안내', 'yes'),
  (9, 'NAVER AI Colloquium 2019 의 참가신청이 완료되었습니다 ', 'yes'),
  (10, ' T아카데미 토크ON세미나 48차 39 Kaggle 캐글 데이터 분석 첫걸음 39 신청 안내', 'yes'),
  (11, 'RE LINE 안녕하세요 라인 채용담당자 이선주 입니다 ', 'yes'),
  (12, ' LINE 안녕하세요 라인 채용담당자 이선주 입니다 ', 'yes'),
  (13, '연구윤리 교육과정 신청 명단 양식입니다 ', 'yes'),
  (14, ' DSC2019 지원서 접수 URL 입니다 ', 'yes')],
 [(15, ' 광고 단하루 대박행사 액츠 액체 세탁세제 선착순 한정특가 ', 'no'),
  (16, ' 광고 명품 브랜드의 러브콜을 부른 기업', 'no'),
  (17, ' 님 혼자만 보기 아까운 트윗을 공유해 보세요 ', 'no'),
  (18, ' 광고 사업자 신규회원 전용 30 쿠폰 2종 쿠폰 혜택 ', 'no'),
  (19, ' 광고 오늘 단하루 커피머신 모음전 파격특가 119 000원 ', 'no'),
 

In [444]:
nYes = len(trainingSetYes)
nNo = len(trainingSetNo)
N = nYes + nNo

priorYes = nYes / N # yes에 대한 prior
priorNo = 1- priorYes
print(priorYes), print(priorNo), print(sum([priorYes, priorNo]))

0.5
0.5
1.0


(None, None, None)

### Trainset에 대한 확률 구하기

In [445]:
# 형태소 분석하기
from nltk.tokenize import sent_tokenize, word_tokenize
from konlpy.corpus import kobill
from collections import defaultdict
from konlpy.tag import Kkma

ma = Kkma()

def ngramUmjeol(term, n = 2): # 음절 단위로 구분하는 함수. sentence를 받아 2개(n=2)씩 쪼갠다.

    ngram = []
    
    for i in range(len(term) - n + 1):
        # ngram.append(tokens_ngram[i:i+n]) # 방법1
        # ngram.append(tuple(tokens_ngram[i:i+n])) # 방법2 (튜플로 반환 시 키값을 쓸 수 있음)
        ngram.append(''.join(term[i:i + n])) # 방법3
        
    return ngram

# 토큰화해주는 함수
def Tokenizer(ListContent):
    dictTerm = list()
    dictPos = list()
    dictNoun = list()
    dictNgram = list()
    th = 1 
    
    for sentence in sent_tokenize(ListContent):
        for token in word_tokenize(sentence):
            if len(token) > th:
                dictTerm.append(token)
                dictPos.extend([morpheme for morpheme in ma.morphs(token) if len(morpheme) > th]) # 형태소 분석결과를 extend
                dictNoun.extend([noun for noun in ma.nouns(token) if len(noun) > th]) # 명사 단위로 잘라 extend
                dictNgram.extend(ngramUmjeol(token)) # 바이그램을 리턴
                
    # 빠른 속도 및 중복 데이터 처리를 위해 set사용 (유일한 단어만 남김)
    dictTerm = tuple(set(dictTerm))
    dictPos = tuple(set(dictPos))
    dictNoun = tuple(set(dictNoun))
    dictNgram = tuple(set(dictNgram))
    dictPosNgram = tuple(set(dictPos+dictNgram))
        
    return dictPosNgram

In [446]:
from collections import defaultdict
tokensYes = defaultdict(int)
tokensNo = defaultdict(int)

for doc in trainingSetYes: # tuple
    for token in Tokenizer(doc[1]): # 단어가 하나씩 추가됨.
        tokensYes[token] += 1
for doc in trainingSetNo: # tuple
    for token in Tokenizer(doc[1]): # token : class 'str'
        tokensNo[token] += 1
        
print(tokensNo), print(tokensYes)
        
# lexicon 생성(set으로 중복 제거 후 list로 묶기)
vocabulary = list(set(list(tokensYes.keys()) + list(tokensNo.keys())))
print(vocabulary)

defaultdict(<class 'int'>, {'대박': 1, '하루': 4, '탁세': 1, '선착순': 2, '정특': 1, '액츠': 1, '선착': 2, '착순': 2, '박행': 1, '단하': 3, '세제': 1, '한정': 1, '행사': 1, '세탁': 1, '광고': 8, '특가': 3, '액체': 1, '브랜드': 1, '명품': 1, '랜드': 1, '부르': 1, '브콜': 1, '러브': 1, '기업': 1, '드의': 1, '브랜': 1, '콜을': 1, '부른': 1, '윗을': 1, '세요': 1, '공유': 1, '보세': 1, '아까': 1, '아깝': 1, '트윗': 1, '혼자': 1, '보기': 1, '자만': 1, '유해': 1, '까운': 1, '업자': 1, '사업자': 1, '회원': 1, '30': 1, '신규': 1, '혜택': 4, '전용': 1, '규회': 1, '사업': 1, '쿠폰': 4, '2종': 1, '00': 3, '119': 1, '머신': 1, '격특': 1, '0원': 3, '11': 2, '피머': 1, '모음': 1, '오늘': 3, '음전': 1, '파격': 1, '19': 1, '000': 1, '커피': 1, '른N': 1, '트코': 1, '코인': 1, '04': 1, '빠르': 1, 'WS': 1, '문가': 1, '3가': 1, '급등': 1, '이유': 1, '비트': 1, '트맨': 1, 'EW': 1, '빠른': 1, 'NE': 1, '꼽은': 1, '가지': 1, 'NEWS': 1, '전문가': 1, '전문': 1, '900': 2, '6입': 1, '21': 2, '24': 1, '90': 2, '4개': 1, '210': 1, '햇반': 1, '10': 2, 'CJ': 1, '플래그': 1, '기즈모입니': 1, '공동': 2, '그쉽': 1, '플래': 1, '모입': 1, '래그': 1, '즈모': 1, '입니': 1, '구매': 2, '제품': 1, '특별'

In [447]:
tokensYes, tokensNo, vocabulary

(defaultdict(int,
             {'메일': 1,
              '구실': 1,
              '00': 1,
              '대표': 1,
              '연구실': 1,
              '17': 1,
              '까지': 1,
              '사요': 1,
              '0까': 1,
              '8월': 1,
              '연구': 3,
              '상주': 1,
              '표연': 1,
              '인원': 1,
              '랩실': 1,
              '조사': 1,
              '연락': 1,
              '요청': 1,
              '학연': 1,
              '표준': 1,
              '포닥': 1,
              '국표': 1,
              '구원': 1,
              '연구원': 1,
              '과학': 1,
              '한국': 1,
              '의뢰': 1,
              '추천': 2,
              '준과': 1,
              '세미나': 2,
              '크O': 2,
              '으로': 1,
              '머신': 1,
              '세미': 2,
              'learn': 1,
              'ci': 2,
              '50': 1,
              'ON': 2,
              '러닝': 1,
              'Scikit': 1,
              '토크': 2,
              '기초': 1,
      

In [448]:
sumYes = sum(tokensYes.values())    # count_tokens_Yes
sumNo = sum(tokensNo.values())    # count_tokens_No
print(sumYes), print(sumNo)

323
243


(None, None)

In [449]:
# conditional probability 계산
from math import log
cpYes = defaultdict(float) # conditional probability이면서 Yes인 것들
cpNo = defaultdict(float) # conditional probability이면서 Yes인 것들
K = 0.5 # K를 통해 스무딩하는 방법
B = len(vocabulary) # 스무딩
sumYes = sum(tokensYes.values())
sumNo = sum(tokensNo.values())

for token in vocabulary:
#     cpYes[token] = log((tokensYes[token] + 1) / (sumYes + B))
#     cpNo[token] = log((tokensNo[token] + 1) / (sumNo + B))
    cpYes[token] = log((tokensYes[token] + K) / (sumYes + K*2))    # 라플라스 스무딩
    cpNo[token] = log((tokensNo[token] + K) / (sumNo + K*2))
    print(token)
    print('Yes, {0} = {1} + 1  / {2} + {3}'.format(cpYes[token], tokensYes[token], sumYes, B))
    print('No, {0} = {1} + 1  / {2} + {3}'.format(cpNo[token], tokensNo[token], sumNo, B))

세미나
Yes, -4.864452783918175 = 2 + 1  / 323 + 406
No, -6.1903154058531475 = 0 + 1  / 243 + 406
명단
Yes, -5.375278407684165 = 1 + 1  / 323 + 406
No, -6.1903154058531475 = 0 + 1  / 243 + 406
데이
Yes, -5.375278407684165 = 1 + 1  / 323 + 406
No, -6.1903154058531475 = 0 + 1  / 243 + 406
iu
Yes, -4.864452783918175 = 2 + 1  / 323 + 406
No, -6.1903154058531475 = 0 + 1  / 243 + 406
드립
Yes, -5.375278407684165 = 1 + 1  / 323 + 406
No, -5.0917031171850375 = 1 + 1  / 243 + 406
사업자
Yes, -6.473890696352274 = 0 + 1  / 323 + 406
No, -5.0917031171850375 = 1 + 1  / 243 + 406
ie
Yes, -5.375278407684165 = 1 + 1  / 323 + 406
No, -6.1903154058531475 = 0 + 1  / 243 + 406
입니
Yes, -4.276666119016055 = 4 + 1  / 323 + 406
No, -5.0917031171850375 = 1 + 1  / 243 + 406
다지
Yes, -5.375278407684165 = 1 + 1  / 323 + 406
No, -6.1903154058531475 = 0 + 1  / 243 + 406
사업
Yes, -6.473890696352274 = 0 + 1  / 323 + 406
No, -5.0917031171850375 = 1 + 1  / 243 + 406
le
Yes, -4.864452783918175 = 2 + 1  / 323 + 406
No, -6.1903154058531

In [450]:
cpYes, cpNo

(defaultdict(float,
             {'세미나': -4.864452783918175,
              '명단': -5.375278407684165,
              '데이': -5.375278407684165,
              'iu': -4.864452783918175,
              '드립': -5.375278407684165,
              '사업자': -6.473890696352274,
              'ie': -5.375278407684165,
              '입니': -4.276666119016055,
              '다지': -5.375278407684165,
              '사업': -6.473890696352274,
              'le': -4.864452783918175,
              '10': -6.473890696352274,
              '었습': -5.375278407684165,
              '관련': -5.375278407684165,
              '유드': -5.375278407684165,
              'om': -5.375278407684165,
              '청이': -5.375278407684165,
              '럭시': -6.473890696352274,
              '메일': -5.375278407684165,
              '러브': -6.473890696352274,
              'NEWS': -6.473890696352274,
              '드리': -5.375278407684165,
              '까운': -6.473890696352274,
              '회원': -6.473890696352274,
              '세

### Testset에 대한 확률 구하기

In [451]:
# 테스트셋 전처리
def Elimpunc_testset(mailTitle):
    patternList = getPatternList()
    for _ in ["Whitespace", "Punctuation", "ElimRecWord"]: 
        mailTitle = patternList[_].sub(" ", mailTitle)
    return mailTitle

In [452]:
from math import exp

i = 0
for doc in testSet:
    scoreYes = log(priorYes)
    scoreNo = log(priorNo)
    Predoc = Elimpunc_testset(doc[1])
    for token in Predoc.lower().split(): # 단어가 하나씩 생김.
        scoreYes += cpYes[token]
        scoreNo += cpNo[token]
        
    # 분류
    print('Yes : {0} / No : {1}'.format(exp(scoreYes), exp(scoreNo)))
    if scoreYes > scoreNo:
        print([testSet[i][1]], "is a [normal mail], the target is", [testSet[i][2]],".") # 정상메일
    else:
        print([testSet[i][1]], "is a [spam mail], the target is", [testSet[i][2]],".") # 스팸메일
    i += 1
    
    # ['200% 정품 비아그라 시알리스 / 온라인 최저가 판매 / 정품아닐시 200% 보상'] is a [normal mail], the target is ['spam'] .의 경우는 
    # training data에 해당 문구가 없기 때문에 오분류 수행.


Yes : 1.190748361530255e-06 / No : 0.0002498488309594196
['(광고) [대학생 복지소식] 삼성 프리도스 노트북 특가 마지막주!'] is a [spam mail], the target is ['spam'] .
Yes : 3.572245084590763e-06 / No : 0.00010707807041117974
["(광고) 신용걱정 끝! 최저금리와 높은승인률로 '최대8천만원'까지 믿음직한 서민신용대출!!"] is a [spam mail], the target is ['spam'] .
Yes : 0.002314814814814814 / No : 0.0010245901639344258
['200% 정품 비아그라 시알리스 / 온라인 최저가 판매 / 정품아닐시 200% 보상'] is a [normal mail], the target is ['spam'] .
Yes : 0.002314814814814814 / No : 0.0010245901639344258
['케라스 & 캐글 전국콘서트 서울 발표자료 송부'] is a [normal mail], the target is ['normal'] .
Yes : 0.002314814814814814 / No : 0.0010245901639344258
['RE: [NAVER] AI 해커톤_AI Vision 오프라인 결선 리마인드'] is a [normal mail], the target is ['normal'] .
Yes : 0.002314814814814814 / No : 0.0010245901639344258
['[한국IT비즈니스진흥협회]인공지능센터 고급\t과정(저녁) 교육생 선정안내'] is a [normal mail], the target is ['normal'] .


###  kNN 문서분류를 Naive Bayes로 변경하여 분류기 만들기